In [ ]:
%load_ext autoreload
%autoreload 2


# Finding heatwaves

According to the World Meteorological Organization (WMO), a heatwave can be defined as "a period where local excess heat accumulates over a sequence of unusually hot days and nights."

In [ ]:
import datetime

import pandas as pd

from meteora import utils
from meteora.clients import GHCNHourlyClient

In [ ]:
region = [8.3424, 47.3416, 8.6734, 47.4673]
# select study period
start_year = 2022
end_year = 2024
# months to consider when querying the GHCNh data
start_month = 5
end_month = 9

# heatwave parameters
heatwave_t_mean_threshold = 25
heatwave_n_consecutive_days = 3

In [ ]:
client = GHCNHourlyClient(region)
ts_df = pd.concat(
    [
        client.get_ts_df(
            "temperature",
            datetime.date(year, start_month, 1),
            # ACHTUNG: this won't work if `end_month` is 12 (see commented code below)
            datetime.date(year, end_month + 1, 1) - datetime.timedelta(days=1),
            # datetime.date(year + int(end_month / 12), (end_month % 12) + 1, 1)
            # - datetime.timedelta(days=1),
        )
        for year in range(start_year, end_year + 1)
    ],
    axis="rows",
)
# convert to wide format
ts_df = utils.long_to_wide(ts_df)

In [ ]:
heatwave_periods = utils.get_heatwave_periods(ts_df, inter_station_agg_func="max")
heatwave_periods

In [ ]:
heatwave_ts_df = (
    pd.concat(
        ts_df.loc[heatwave_start:heatwave_end].assign(
            heatwave=f"{heatwave_start.date()}/{heatwave_end.date()}"
        )
        for heatwave_start, heatwave_end in heatwave_periods
    )
    .reset_index()
    .set_index(["heatwave", ts_df.index.name])
)
heatwave_ts_df

In [ ]:
heatwave_ts_df.reset_index().melt(
    id_vars=["heatwave", "time"], value_name="temperature"
)

## References

1. A. Burgstall, L. Fontannaz, M. Gaia, R. Gehrig, U. Graf, S. Gubler, S. Kotlarski, D. Murer, T. Schlegel, C. Schwierz, and C. Spirig, “Von den warnungen bis zur klimatologie – das thema hitze an der MeteoSchweiz,” Technical report MeteoSwiss No. 276 276, MeteoSwiss, 2021.